# Laboratorium 5 - rekomendacje grafowe

## Przygotowanie

 * dataset i potrzebne biblioteki są dokładnie takie same jak na poprzednim laboratorium
 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab5`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas sklearn gensim==3.8.3`

## Część 1. - przygotowanie danych

In [1]:
# importujemy wszystkie potrzebne pakiety

import math
import random
import numpy as np
import pandas

from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split, KFold

In [2]:
SCORE_THRESHOLD = 4.0 # recenzje z co najmniej taka ocena wezmiemy pod uwage
VECTOR_SIZE = 20 # jak dlugie powinny byc wektory osadzen wierzcholkow
NEIGHBOURS_WINDOW = 11 # tylu sasiadow wezmiemy pod uwage w algorytmie Word2Vec (symetrycznie i wliczajac biezacy element)
PATH_LENGTH = 30 # dlugosc pojedynczej losowej sciezki
PATHS_COUNT_PER_NODE = 20 # liczba losowych sciezek zaczynajacych sie w kazdym z wierzcholkow

In [3]:
# wczytujemy oceny uytkownikow

ratings = pandas.read_csv('ratings.csv').drop(columns=['timestamp'])
ratings = ratings.where(ratings['rating'] >= SCORE_THRESHOLD).dropna()
# rozszerzamy ID tak, by sie nie powtarzaly
ratings['userId'] = ratings['userId'].apply(lambda x: 'u_' + str(int(x)))
ratings['movieId'] = ratings['movieId'].apply(lambda x: 'm_' + str(int(x)))
ratings

,userId,movieId,rating
0,u_1,m_1,4.0
1,u_1,m_3,4.0
2,u_1,m_6,4.0
3,u_1,m_47,5.0
4,u_1,m_50,5.0
...,...,...,...
85263,u_554,m_2087,4.0
85264,u_554,m_2096,4.0
85267,u_554,m_2282,5.0
85268,u_554,m_2324,5.0


In [4]:
# wczytujemy gatunki filmow

movies = pandas.read_csv('movies.csv').drop(columns=['title'])
movies['movieId'] = movies['movieId'].apply(lambda x: 'm_' + str(int(x)))
movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))
movies_to_genres = movies.explode('genres')
movies_to_genres['genres'] = movies_to_genres['genres'].apply(lambda x: 'g_' + x.lower())
movies_to_genres = movies_to_genres.rename(columns = {'genres': 'genre'})
movies_to_genres

,movieId,genre
0,m_1,g_adventure
0,m_1,g_animation
0,m_1,g_children
0,m_1,g_comedy
0,m_1,g_fantasy
...,...,...
9738,m_193583,g_fantasy
9739,m_193585,g_drama
9740,m_193587,g_action
9740,m_193587,g_animation


In [47]:
users = ratings['userId'].unique()
movies = movies_to_genres['movieId'].unique()
genres = movies_to_genres['genre'].unique()
print(genres)

['g_adventure' 'g_animation' 'g_children' 'g_comedy' 'g_fantasy'
 'g_romance' 'g_drama' 'g_action' 'g_crime' 'g_thriller' 'g_horror'
 'g_mystery' 'g_sci-fi' 'g_war' 'g_musical' 'g_documentary' 'g_imax'
 'g_western' 'g_film-noir' 'g_(no genres listed)']


## Część 2. - spacer po grafie

In [7]:
from collections import defaultdict
from random import random, choice
import random

def generate_walks(ratings, movies_to_genres, paths_per_node, path_length):
    paths = []
    memory = defaultdict(list)
    for idx, node in enumerate(np.concatenate([users, movies, genres])):
        for _ in range(PATHS_COUNT_PER_NODE):
            path = []
            for _ in range(PATH_LENGTH):
                if node[0] == 'm':
                    if memory[node]:
                        sample = random.choice(memory[node])
                    else:
                        m_to_g = movies_to_genres.loc[movies_to_genres['movieId'] == node, 'genre'].to_list()
                        m_to_u = ratings.loc[ratings['movieId'] == node, 'userId'].to_list()
                        u_g = m_to_g + m_to_u
                        sample = random.choice(u_g)
                        memory[node] = u_g
                elif node[0] == 'u':
                    if memory[node]:
                        sample = random.choice(memory[node])
                    else:
                        m = ratings.loc[ratings['userId'] == node, 'movieId'].to_list()
                        sample = random.choice(m)
                        memory[node] = m
                else:
                    if memory[node]:
                        sample = random.choice(memory[node])
                    else:
                        g = movies_to_genres.loc[movies_to_genres['genre'] == node, 'movieId'].to_list()
                        sample = random.choice(g)
                        memory[node] = g
                    
                path.append(sample)
                node = sample
                    
            paths.append(path)
            
        if idx % 100 == 0:
            print(idx)
            
        
    return paths
    
walks = generate_walks(ratings, movies_to_genres, PATHS_COUNT_PER_NODE, PATH_LENGTH)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300


In [6]:
# generujemy losowe sciezki w grafie
#   krawedzie reprezentowane sa w dwoch macierzach - ratings i movies
#   w wersji podstawowej wszystkie krawedzie traktujemy jako niewazone i nieskierowane
#   mozliwe ulepszenia:
#    - rozwazenie krawedzi skierowanych
#    - uwzglednienie wag krawedzi (ocen uzytkownikow)
#    - jakas forma normalizacji - obnizenia wag wierzcholkow o wysokich stopniach
#    - Node2Vec - parametry P i Q
# wynikiem powinna byc lista list - kazda z tych list zawiera kolejne ID wierzcholkow na sciezce
from enum import Enum
from random import random, choice

class Prev(Enum):
  User = 1
  Movie = 2
  Genre = 3

def my_where(arr, axis, to_find):
    a = None
    for val in arr:
      # print(val, val[axis], to_find, val[axis] == to_find)
      # print(a)
      if val[axis] == to_find:
        if a is not None:
          a = np.append(a, np.array([val]), axis=0)
        else:
          a = np.array([val])
    # print(a)
    return a

def get_paths(node_name, prev, ratings, movies_to_genres, paths_per_node, path_length):
    paths = []
    for path_number in range(paths_per_node):
      path = [node_name]  
      for path_step in range(path_length):
        # print(f"nodename_main: {node_name}")
        if prev == Prev.User:
          print(f"nodename_user: {node_name}")
          # print(ratings[:,0])
          # possible_nodes = my_where(ratings, 0, node_name)
          possible_nodes = np.where(ratings[:,0] == node_name)
          # print(ratings[possible_nodes])
          # next_node = choice(possible_nodes)[1]
          next_node = choice(ratings[possible_nodes])[1]
          prev = Prev.Movie
        elif prev == Prev.Movie:
          print(f"nodename_movie: {node_name}")
          if random() < .5: #go to user
            # possible_nodes = my_where(ratings, 1, node_name)
            possible_nodes = np.where(ratings[:,1] == node_name)
            # print(ratings[possible_nodes])
            # print(choice(ratings[possible_nodes])[0])
            # next_node = choice(possible_nodes)[0]
            next_node = choice(ratings[possible_nodes])[0]
            prev = Prev.User
          else: #go to genre
            # possible_nodes = my_where(movies_to_genres, 0, node_name)
            possible_nodes = np.where(movies_to_genres[:,0] == node_name)
            next_node = choice(movies_to_genres[possible_nodes])[1]
            prev = Prev.Genre
        elif prev == Prev.Genre:
          print(f"nodename_genre: {node_name}")
          # possible_nodes = my_where(movies_to_genres, 1, node_name)
          possible_nodes = np.where(movies_to_genres[:,1] == node_name)
          next_node = choice(movies_to_genres[possible_nodes])[0]
          prev = Prev.Movie        
        node_name = next_node
        path.append(next_node)
      paths.append(path)
    return paths



def generate_walks(ratings, movies_to_genres, paths_per_node, path_length):
    paths = []
    # print(f"ratings: \n{ratings}")
    # print(f"movies to genres: \n{movies_to_genres}")
    # print(f"pathspernode: \n{paths_per_node}")
    # print(f"path_length: \n{path_length}")
    # print(f"unique users: {ratings['userId'].unique()}")
    # print(f"unique movies: {ratings['movieId'].unique()}")
    # print(f"unique genres: {movies_to_genres['genre'].unique()}")
    ratings = ratings.to_numpy()
    movies_to_genres = movies_to_genres.to_numpy()
    
    possible_nodes = np.where(ratings[:,1] == 'm_108090')
    print(ratings[possible_nodes])
    for i in ratings:
      if i[1] == 'm_108090':
        print(i)
    for user_id in users:
      path = get_paths(user_id, Prev.User, ratings, movies_to_genres, paths_per_node, path_length)
      paths.extend(path)
    for movie_id in movies:
      path = get_paths(movie_id, Prev.Movie, ratings, movies_to_genres, paths_per_node, path_length)
      paths.extend(path)
    for genre in genres:
      path = get_paths(genre, Prev.Genre, ratings, movies_to_genres, paths_per_node, path_length)
      paths.extend(path)

    # ...
    return paths
    
walks = generate_walks(ratings, movies_to_genres, PATHS_COUNT_PER_NODE, PATH_LENGTH)

[]
nodename_user: u_1
nodename_movie: m_3439
nodename_genre: g_fantasy
nodename_movie: m_2193
nodename_user: u_527
nodename_movie: m_3359
nodename_user: u_527
nodename_movie: m_4638
nodename_genre: g_sci-fi
nodename_movie: m_87306
nodename_user: u_29
nodename_movie: m_2028
nodename_user: u_79
nodename_movie: m_3006
nodename_user: u_339
nodename_movie: m_174053
nodename_genre: g_sci-fi
nodename_movie: m_33004
nodename_genre: g_adventure
nodename_movie: m_7070
nodename_user: u_474
nodename_movie: m_3793
nodename_genre: g_action
nodename_movie: m_99112
nodename_user: u_119
nodename_movie: m_60069
nodename_genre: g_children
nodename_movie: m_58105
nodename_genre: g_imax
nodename_movie: m_95105
nodename_user: u_210
nodename_movie: m_90866
nodename_genre: g_mystery
nodename_movie: m_1783
nodename_user: u_465
nodename_movie: m_2763
nodename_genre: g_mystery
nodename_movie: m_90866
nodename_genre: g_mystery
nodename_movie: m_942
nodename_user: u_410
nodename_movie: m_1211
nodename_user: u_275


IndexError: ignored

## Część 3. - obliczenie osadzeń

In [8]:
# trenujemy model
#   zauwaz, ze wszystkie trzy rodzaje wierzcholkow beda reprezentowane tak samo, w tej samej przestrzeni

model = Word2Vec(sentences=walks, size=VECTOR_SIZE, window=NEIGHBOURS_WINDOW, min_count=1, workers=4)
embeddings = model.wv

[('u_234', 0.8150248527526855),
 ('u_561', 0.7639709711074829),
 ('u_135', 0.7418652772903442),
 ('u_27', 0.7302687168121338),
 ('u_294', 0.7269852757453918)]

## Część 4. - rekomendacje i zastosowania

In [9]:
PULP_FICTION = 'm_296'
TOY_STORY = 'm_1'
PLANET_OF_THE_APES = 'm_2529'

In [56]:
# wyszukajmy K najpodobniejszych filmów do danego
# porownaj wyniki dla odleglosci euklidesowej i cosinuswej, np. na trzech powyzszych filmach
from math import sqrt

def euclidian_distance(i, j):
    return sqrt(sum(np.square(i-j)))

def cosine_distance(i, j):
    return -1 * sum(np.multiply(i, j)) / sqrt(sum(np.square(i)) * sum(np.square(j)))

def k_most_similar_movies(movie_id, K, embeddings, distance_fun):
    # ...
    similars = sorted([(movie, distance_fun(embeddings[movie_id], embeddings[movie])) for movie in movies], key=lambda x: x[1])
    return similars[1:K+1]

def k_most_similar_default(id, K, embeddings):
  return embeddings.most_similar(id, topn=K)

print(k_most_similar_default(PULP_FICTION, 5, embeddings))
print(k_most_similar_movies(PULP_FICTION, 5, embeddings, cosine_distance))
print(k_most_similar_movies(PULP_FICTION, 5, embeddings, euclidian_distance))

[('m_50', 0.9878280162811279), ('m_593', 0.9855232238769531), ('m_47', 0.9830543994903564), ('m_318', 0.976776659488678), ('m_527', 0.9681359529495239)]
[('m_50', -0.9878280568192744), ('m_593', -0.9855232213062911), ('m_47', -0.9830543612470516), ('m_318', -0.9767766879371755), ('m_527', -0.9681359997800218)]
[('m_50', 4.008632444744372), ('m_593', 4.163454231067091), ('m_47', 4.798447874463489), ('m_318', 5.432245129440272), ('m_527', 6.151622343910765)]


In [32]:
# wyszukajmy k filmow najblizszych uzytkownikowi
# wykorzystaj funkcje z poprzedniej komorki

def k_best_movies_for_user(user_id, K, embeddings, distance_fun):
    # ...
    similars = sorted([(movie, distance_fun(embeddings[user_id], embeddings[movie])) for movie in movies], key=lambda x: x[1])
    return similars[:K]

print(k_best_movies_for_user('u_1', 10, embeddings, cosine_distance))
print(k_best_movies_for_user('u_1', 10, embeddings, euclidian_distance))
# print(k_most_similar_movies_default('u_1', 10, embeddings,))

[('m_2427', -0.2254094609056643), ('m_1275', -0.13041081794168313), ('m_316', -0.12225793493988774), ('m_1049', -0.1179877806743782), ('m_610', -0.11012599534707553), ('m_2872', -0.1050833115954454), ('m_1343', -0.09748110227014806), ('m_553', -0.08033334358452857), ('m_514', -0.07698415770680116), ('m_671', -0.07696920259028808)]
[('m_305', 21.540233757321065), ('m_4866', 21.74527202936019), ('m_2907', 21.778537113524692), ('m_2534', 21.806268687307192), ('m_2965', 21.815754418125156), ('m_63808', 21.828987175551276), ('m_27830', 21.85425707856257), ('m_2644', 21.856609626289753), ('m_2526', 21.873792209660817), ('m_5358', 21.886181322478382)]


In [44]:
genre_to_movies = {}
movies_to_genres_np = movies_to_genres.to_numpy()
for movie in movies:
  genres2 = []
  for item in movies_to_genres_np:
    if item[0] == movie:
      genres2.append(item[1])
  # genres = movies_to_genres_np[np.where(movies_to_genres_np[0]==movie)]
  # print(movies_to_genres_np)
  # print(movie)
  # print(movies_to_genres_np[np.where(movies_to_genres_np[0]==movie)])
  for genre in genres2:
    try:
      genre_to_movies[genre].append(movie)
    except:
      genre_to_movies[genre] = [movie]
print(genre_to_movies)

{'g_adventure': ['m_1', 'm_2', 'm_8', 'm_10', 'm_13', 'm_15', 'm_29', 'm_44', 'm_53', 'm_60', 'm_86', 'm_95', 'm_101', 'm_107', 'm_112', 'm_126', 'm_146', 'm_150', 'm_153', 'm_155', 'm_158', 'm_160', 'm_169', 'm_170', 'm_208', 'm_212', 'm_231', 'm_238', 'm_258', 'm_260', 'm_316', 'm_329', 'm_340', 'm_362', 'm_364', 'm_368', 'm_380', 'm_393', 'm_421', 'm_432', 'm_434', 'm_442', 'm_455', 'm_459', 'm_464', 'm_480', 'm_484', 'm_485', 'm_494', 'm_533', 'm_546', 'm_547', 'm_552', 'm_558', 'm_577', 'm_588', 'm_590', 'm_599', 'm_609', 'm_610', 'm_631', 'm_648', 'm_653', 'm_661', 'm_673', 'm_674', 'm_688', 'm_704', 'm_709', 'm_711', 'm_720', 'm_733', 'm_736', 'm_761', 'm_780', 'm_798', 'm_808', 'm_809', 'm_828', 'm_836', 'm_849', 'm_888', 'm_897', 'm_908', 'm_919', 'm_924', 'm_940', 'm_941', 'm_952', 'm_969', 'm_970', 'm_986', 'm_990', 'm_1008', 'm_1009', 'm_1015', 'm_1017', 'm_1019', 'm_1027', 'm_1030', 'm_1031', 'm_1032', 'm_1049', 'm_1085', 'm_1127', 'm_1129', 'm_1136', 'm_1196', 'm_1197', '

In [66]:
# sprobujmy czegos bardziej skomplikowanego
#   znajdz ulubiony gatunek filmowy uzytkownika
#   a nastepnie zaproponuj K filmow z tego gatunku - ale nie tych najblizszych uzytkownikowi
#   (zaproponuj, w jaki sposob dobrac filmy interesujace, ale nie z najblizszego otoczenia)

def k_from_favourite_genre(user_id, K, embeddings, distance_fun):
    # ...
    fav_genre = sorted([(genre, distance_fun(embeddings[user_id], embeddings[genre])) for genre in genres], key=lambda x: x[1])[0][0]
    movies_from_genre = genre_to_movies[fav_genre]
    sorted_movies = sorted([(movie, distance_fun(embeddings[user_id], embeddings[movie])) for movie in movies_from_genre], key=lambda x: x[1])[:5*K]
    proposed_movies = set()
    while len(proposed_movies) < K:
      proposed_movies.add(choice(sorted_movies))
    return list(proposed_movies)

    # return k_from_genre
  
k_from_favourite_genre('u_100', 5, embeddings, cosine_distance)

[('m_168', -0.2648273895209445),
 ('m_1057', -0.26342850384263133),
 ('m_2485', -0.262358534016865),
 ('m_440', -0.2458556180054678),
 ('m_207', -0.2888664950191855)]

In [74]:
# Na koniec najbardziej skomplikowany algorytm - odpowiednik "Radia utworu" w Spotify.
#   Zaczynamy od jednego filmu, a nastepnie wyznaczamy kolejne, wedrujac po przestrzeni, w ktorej wszystkie elementy sa osadzone.
#   Zaproponuj, jak zdefiniowac podzbior filmow, z ktorych bedziemy wybierac (np. filmy odlegle o min. a i max. b od danego)
#   oraz jak generowac kolejny skok (tak, zeby seria rekomendacji nie byla zbyt monotonna, ale rownoczesnie zgodna z gustem uzytkownika)

# Będą trzy przypadki, 
#     1: losuje następny film który jest podobny do danego filmu
#     2: losuje film ulubionej kategorii (upgrade: wybieram podobną kategorię dla danego filmu i z tych filmów wybieram rekomendację )
#     3: losuję film który spodobał się użytkownikowi który jest podobny do użytkownika głównego

def get_playlist(start_movie_id, user_id, K, embeddings):
    playlist = set()
    dist_func = cosine_distance
    movie = start_movie_id
    while len(playlist) < K:
        randed = random.random()
        if randed < .3333333: #1
          movie = choice(k_most_similar_movies(movie, 5, embeddings, dist_func))[0]
        elif randed < .6666666: #2
          movie = choice(k_from_favourite_genre(user_id, 5, embeddings, dist_func))[0]
        else: #3
          similar_user = choice(k_most_similar_default(user_id, 10, embeddings))[0]
          movie = choice(k_best_movies_for_user(similar_user, 10, embeddings, dist_func))[0]
        playlist.add(movie)
    # ...
    return playlist
  
get_playlist(PULP_FICTION, 'u_100', 10, embeddings)


{'m_1057',
 'm_11',
 'm_135',
 'm_1414',
 'm_1629',
 'm_195',
 'm_207',
 'm_3512',
 'm_376',
 'm_539'}